# Creating Cutouts of Roman Data with Astrocut

---

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## IMPORTANT: Astrocut Version

Be sure to upgrade the astrocut version to 1.0.1 before running this tutorial. Check it now:

In [ ]:
import astrocut

print(f'astrocut version = {astrocut.__version__}')

**If you have astrocut version 1.0.0**, then uncomment the following cell, run it, and then restart the kernel. You can restart the kernel by going to the menubar, selecting "Kernel" then "Restart Kernel...". You can also press the button at the top of the notebook that looks like a circle with an arrow.

**Failure to upgrade astrocut to version 1.0.1 will result in errors in the tutorial notebook cells.**

In [ ]:
#!pip install --upgrade astrocut==1.1.0

## Imports

The following packages are used for these reasons:

- *asdf* - To handle ASDF input and output
- *roman_datamodels* - To read Roman WFI ASDF files
- *s3fs* - To access cloud files as though they were local
- *matplotlib* - The plotting package used for demonstrating the cutout
- *numpy* - To handle array functions
- *astrocut* - The main package import, used to generate the cutouts from the input file
- *astropy* - To manage the output FITS data file type and handle input sky coordinates

In [ ]:
import warnings
from copy import deepcopy

import asdf
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import roman_datamodels as rdm
import s3fs
from astrocut import get_center_pixel, ASDFCutout
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.utils.exceptions import AstropyDeprecationWarning

# Ignore warnings from the 'astropy.wcs' module
# When creating cutouts -> WARNING: Polynomial distortion is not implemented.
warnings.filterwarnings('ignore', module='astropy.wcs')

# Ignore AstropyDeprecationWarning
# When creating cutouts -> WARNING: AstropyDeprecationWarning: The class "Fits" has been renamed to "FITS" in version 7.0. 
# The old name is deprecated and may be removed in a future version. Use FITS instead. [astropy.units.format]
warnings.filterwarnings('ignore', category=AstropyDeprecationWarning)

## Introduction

The `astrocut` tool allows users to generate cutouts out of large astronomical images. In this tutorial, we demonstrate how to use `astrocut` to generate cutouts from a calibrated Level 2 (L2; calibrated rate image) simulated product created with Roman I-Sim.

---

## Setup

The first step of the analysis is to read the Roman WFI image data, which are stored in Advanced Scientific Data Format (ASDF) files. For this example, we start with a calibrated L2 simulated image created with [Roman I-Sim](https://romanisim.readthedocs.io/en/latest/). For more information about Roman's data products, check the [Roman User Documentation](https://roman-docs.stsci.edu/).

After launch, Roman data will be available through the [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/). For testing purposes, some simulated data has been placed in a separate S3 bucket.  We access the Roman data files from the RRN S3 bucket using `s3fs`. For more information about how to access the S3 bucket, please see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

In [ ]:
fs = s3fs.S3FileSystem(anon=True)
asdf_dir_uri = 's3://stpubdata/roman/nexus/soc_simulations/tutorial_data/'
asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi02_f106_cal.asdf'

# Read the file into memory
with fs.open(asdf_file_uri, 'rb') as f:
    with asdf.open(f) as af:
        dm = rdm.open(af)
        data = deepcopy(dm.data)
        gwcs = deepcopy(dm.meta.wcs)

## Examine Coordinates

### GWCS

Roman ASDF images use a [Generalized World Coordinate System (GWCS)](https://gwcs.readthedocs.io/en/latest/index.html) object to handle coordinate transformations, as opposed to the [Flexible Image Transport System (FITS)](https://fits.gsfc.nasa.gov/fits_primer.html) [WCS standard](https://fits.gsfc.nasa.gov/fits_wcs.html). GWCS is more flexible than FITS WCS, and contains a compound model of transformations from detector to world coordinates.  It is stored in the image files within the `meta.wcs` attribute.  Let's examine the sky and pixel coordinates of our image:

In [ ]:
print('Some GWCS Info:')
print('---------------')
print('- Input frame: ', gwcs.input_frame)
print('- Output frame: ', gwcs.output_frame)
print('- Pixel bounds: ', gwcs.bounding_box)
print('- Sky footprint: ', gwcs.footprint())

### Coordinate Analysis

Next, we will do some coordinate analysis of points in the image. For three sets of input sky coordinates, we will compute the pixel coordinates, the computed sky coordinates, and the separation between the input and computed sky coordinates. The points correspond to the center of the image, the far upper right, and the far lower left. First, let's define some helper functions:

In [ ]:
# Define some helper functions

def get_sep(coord1, coord2):
    """ Compute separation between two sky coordinates """
    return coord1.separation(coord2).to(u.arcsec)
    

def get_pix_info(xx, coord):
    """ Print some pixel coordinate info """
    print('Pixel Coord:', xx)
    ss = gwcs.pixel_to_world(*xx)
    print('Computed Sky Coord:', ss)
    print('Separation:', get_sep(coord, ss))    


def print_coord_info(coord):
    """ Print some sky coordinate info """
    print('Input Sky Coord:', coord)
    xx = gwcs.invert(coord)
    get_pix_info(xx, coord)

Next, we will do some coordinate analysis for the center of the image, the far upper-right corner, and the far lower-left corner in the image. For three sets of input sky coordinates, we will compute the pixel coordinates, the computed sky coordinates, and the separation between the input and computed sky coordinates.

In [ ]:
# Sky coord of central pixel
print('Central Image Pixel')
print('-------------------')
pix_x, pix_y = np.shape(data)[0] // 2, np.shape(data)[1] // 2
center_coord = gwcs.pixel_to_world(pix_x, pix_y)
print_coord_info(center_coord)
print('\n')

# Sky coord at far edge (upper right)
print('Far Edge Pixel - Upper Right')
print('-----------------------------')
ss = gwcs.pixel_to_world(4077, 4077)
print_coord_info(ss)
print('\n')

# Sky coord at far edge (lower left)
print('Far Edge Pixel - Lower Left')
print('---------------------------')
ss = gwcs.pixel_to_world(2, 2)
print_coord_info(ss)
print('\n')

# Get computed pixel coordinates from world coordinates with get_center_pixel()
test_coord = SkyCoord('270.8719 -0.26437', unit='deg') # test out different sky coordinates here!
print('Computed Closet Pixel Coordinate')
print('--------------------------------')
print('Test Sky Coord:', test_coord)
pc, ww = get_center_pixel(gwcs, test_coord.ra, test_coord.dec)
get_pix_info(pc, test_coord)


---

## Creating the ASDF Image Cutout 
Here, we will create the image cutout.  First, we plot the expected cutout on the input image with a red square.

In [ ]:
# Define cutout size
cutout_size = 200

# Define cutout coordinates
coord = center_coord

# Plot original image
plt.imshow(data, vmin=0, vmax=5, origin='lower')

# Plot the expected cutout with red square
pc, ww = get_center_pixel(gwcs, coord.ra, coord.dec)
tmp = Cutout2D(data, position=pc, wcs=ww, size=cutout_size, mode='partial')
tmp.plot_on_original(color='red')

# More plot details
plt.title('Expected Cutout Position')
padding = 500
plt.xlim(pix_x - padding, pix_x + padding)
plt.ylim(pix_y - padding, pix_y + padding)
plt.show()

The `~astrocut.ASDFCutout` class takes as input the following: 
- `input_files`: Input ASDF filename(s) or URI(s)
- `coordinates`: The coordinates of the cutout center.
- `cutout_size`: The size of the cutout array in pixels or angular units.
- `fill_value`: The value for any cutout pixels outside the original image (optional, default `np.nan`).
- `verbose`: If `True`, log messages are printed to the console (optional, default `False`).
  
Let's create a square cutout at the center of the image with 200 pixels on each side.

In [ ]:
# Create the image cutout
asdf_cutout = ASDFCutout(asdf_file_uri, coord, cutout_size)

The resulting `ASDFCutout` object can be used to access the cutout data and metadata. The following attributes are available:
- `cutouts`: A list of cutouts as `~astropy.nddata.Cutout2D` objects. These objects contain the cutout data, shape, world coordinate system, and other helpful properties.
- `asdf_cutouts`: A list of cutouts as `~asdf.AsdfFile` objects.
- `fits_cutouts`: A list of cutouts as `~astropy.io.fits.HDUList` objects.

The cutout objects in these lists can be used to access cutout data and metadata, as shown in the cells below.

In [ ]:
# Access ~astropy.nddata.Cutout2D object
cutout = asdf_cutout.cutouts[0]
print('Cutout Data Shape:', cutout.shape)
print('Cutout Original Position:', cutout.position_original)
print('Cutout Position:', cutout.position_cutout)

In [ ]:
# Access ~asdf.AsdfFile object
cutout_asdf = asdf_cutout.asdf_cutouts[0]
cutout_asdf.info()

In [ ]:
# Access ~astropy.io.fits.HDUList object
cutout_fits = asdf_cutout.fits_cutouts[0]
cutout_fits.info()

### Inspect ASDF Cutout Object

Now, let's inspect the ASDF cutout object and display the cutout with matplotlib.

In [ ]:
data_asdf = cutout_asdf['roman']['data']
gwcs_asdf = cutout_asdf['roman']['meta']['wcs']

#### Plot Cutout in Pixel Coordinates

Next, we will plot the cutout in pixel coordinates. The pixel coordinates are shown along the axes, and the cutout is a 200 x 200 pixel square.

In [ ]:
def plot_cutout_pixel(img):
    """ Display the cutout using pixel coordinates """
    plt.figure(figsize=(6, 6))
    plt.imshow(img, vmin=0, vmax=5, origin="lower")
    plt.title('Cutout in Pixel Coordinates')
    plt.show()


plot_cutout_pixel(data_asdf)

#### Plot Cutout in Sky Coordinates

Next, we will plot the cutout in sky coordinates. We will use the cutout's GWCS object as a plot projection.

In [ ]:
def plot_cutout_world(img, coord, wcs):    
    """ Display the cutout using world coordinate system """
    plt.figure(figsize=(6, 6))
    plt.title('Cutout in Sky Coordinates')
    plt.xticks([])
    plt.yticks([])
    ax = plt.subplot(projection=wcs)
    ax.imshow(img, vmin=0, vmax=5, origin='lower')
    ax.grid(color='white', ls='solid')
    
    # Overplot the original sky coordinate in cyan
    ax.scatter_coord(coord, s=100, edgecolor='cyan', facecolor='none')
    
    # Plot central pixel in white
    center_loc = img.shape[0] // 2
    ax.scatter([center_loc], [center_loc], s=100, edgecolor='white', facecolor='none')

    # Plot sky coord of central pixel in yellow
    if hasattr(wcs, 'all_pix2world'):
        # WCS case
        ra, dec = wcs.all_pix2world(center_loc, center_loc, 0)
    else:
        # GWCS case
        ra, dec = wcs(center_loc, center_loc)
    cc = SkyCoord(ra, dec, unit='deg')
    ax.scatter_coord(cc, s=50, edgecolor='yellow', facecolor='none')
    
    plt.show()

In [ ]:
# Print the cutout GWCS
print(gwcs_asdf, '\n')

# Print the position of input coordinates
print("SkyCoords Info:")
print('Input Requested', coord)

# Plot the figure with WCS
header_gwcs = gwcs_asdf.to_fits_sip()
wcs = WCS(header_gwcs)
plot_cutout_world(data_asdf, coord, wcs)

### Inspect FITS Cutout Object

Now, let's inspect the FITS cutout object and display the cutout with matplotlib.

In [ ]:
data_fits = cutout_fits[0].data
header = cutout_fits[0].header
wcs_fits = WCS(header)

#### Plot Cutout in Pixel Coordinates

Next, we will plot the cutout in pixel coordinates. The pixel coordinates are shown along the axes, and the cutout is a 200 x 200 pixel square.

In [ ]:
plot_cutout_pixel(data_fits)

#### Cutout in Sky Coordinates

Next, we will plot the cutout in sky coordinates. We will use the cutout's GWCS object as a plot projection.

In [ ]:
# Print the cutout WCS
print(wcs, '\n')

# Print the position of input coordinates
print("SkyCoords Info:")
print('Input Requested', coord)

# Plot the figure with WCS
plot_cutout_world(data_fits, coord, wcs)

---

## Partial Image Cutouts

Sometimes, cutouts are made near the edge of Roman images. In these cases, part of the cutout may fall outside of the original image.

The `ASDFCutout` class takes an optional `fill_value` parameter for this circumstance. It describes the value that should be given to any pixels in the cutout that fall outside of the input image. The default value is `np.nan`.

To illustrate this, we will take a cutout from the lower right corner of the image. The cutout will be 300 pixels on each side.

In [ ]:
# Roman images are 4088 x 4088, so pick values for edge_x and edge_y that are greater than 4088 - cutout_size
cutout_size = 300
edge_x = 4000
edge_y = 4000

# Convert to sky coordinates
edge_coord = gwcs.pixel_to_world(edge_x, edge_y)
edge_coord

# Plot original image
plt.imshow(data, vmin=0, vmax=5, origin='lower')

# Plot the expected cutout with red square
pc, ww = get_center_pixel(gwcs, edge_coord.ra, edge_coord.dec)
tmp = Cutout2D(data, position=pc, wcs=ww, size=cutout_size, mode='partial')
tmp.plot_on_original(color='red')

# More plot details
plt.title('Expected Cutout Position')
padding = 500
plt.xlim(edge_x - padding, edge_x + padding)
plt.ylim(edge_y - padding, edge_y + padding)
plt.show()

Notice how the top and right side of the square are outside of the image? We can assign the value of these pixels in our cutout using the `fill_value` parameter.

### Create Image Cutout

We will create the partial cutout and fill in any outside pixels with a value of 0.

In [ ]:
# Create the partial image cutout
asdf_cutout_partial = ASDFCutout(asdf_file_uri, edge_coord, cutout_size, fill_value=0)

# Get Cutout2D object
cutout_partial = asdf_cutout_partial.cutouts[0]
print('Cutout Data Shape:', cutout_partial.shape)
print('Cutout Original Position:', cutout_partial.position_original)
print('Cutout Position:', cutout_partial.position_cutout)

### Inspect Image Cutout

Let's inspect the cutout object and display it with matplotlib.

In [ ]:
# Plot cutout in pixel coordinates
plot_cutout_pixel(cutout_partial.data)

In [ ]:
# Print the cutout wcs
print(cutout_partial.wcs, '\n')

# Print FITS WCS position of various sky coordinates
print("SkyCoords Info:")
print('Input Requested', edge_coord)

# Plot the figure with wcs
plot_cutout_world(cutout_partial.data, edge_coord, cutout_partial.wcs)

As expected, the pixels that fall outside of the original image have been assigned a value of 0.

---

## Additional Resources
- [Astrocut Documentation](https://astrocut.readthedocs.io)
- [Advanced Scientific Data Format Documentation](https://asdf.readthedocs.io/en/latest/)

---

## About this Notebook

**Authors**: Thomas Dutkiewicz, Brian Cherinka, Sam Bianco<br>
**Last Updated**: 2025-06-05

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 